<a href="https://colab.research.google.com/github/kuldeepthakare7/Chirp-Birdsong-recognition/blob/main/Neural_Network_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import librosa
import librosa.display
import soundfile as sf
import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical

In [ ]:
df  = pd.read_csv('/content/UrbanSound8K.csv')

In [ ]:
def feature_extractor(filename):
  Audio,sr = librosa.load(filename)
  mel_signal = librosa.feature.melspectrogram(y=Audio, sr=sr, n_fft=1024, hop_length=64)
  mel_spect = librosa.power_to_db(np.abs(mel_signal), ref=np.max)
  mel_scaled = np.mean(mel_spect.T,axis=0)
  return mel_scaled

In [ ]:
labels = list(df['class'].unique())
files = dict()
for i in range(len(labels)):
    tmp = df[df['class'] == labels[i]][:1].reset_index()
    path = 'UrbanSound8K/audio/fold{}/{}'.format(tmp['fold'][0], tmp['slice_file_name'][0])
    files[labels[i]] = path

In [ ]:
print(files)

In [ ]:
features=[]
labels =[]
for index,row in df.iterrows():
  file_name = row['slice_file_name']
  class_label = row['class']
  features.append(feature_extractor(file_name))
  labels.append(class_label)


In [ ]:
featuresdf = pd.DataFrame(features, columns=['feature', 'fold' ,'class_label'])

In [ ]:
train = featuresdf[featuresdf['fold'] <= 8]
test = featuresdf[featuresdf['fold'] > 8]
x_train = np.array(train.feature.tolist())
y_train = np.array(train.class_label.tolist())
x_test = np.array(test.feature.tolist())
y_test = np.array(test.class_label.tolist())
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y_train))
y_test = to_categorical(le.fit_transform(y_test))

In [ ]:
num_labels = y_train.shape[1]
filter_size = 2
def build_model_graph(input_shape=(40,)):
    model = Sequential()
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model
model = build_model_graph()

In [ ]:
from keras.callbacks import ModelCheckpoint
from datetime import datetime
num_epochs = 100
num_batch_size = 32
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)

In [ ]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))